# Batch Gender Detection and Image Processing

Processes a batch of images in a specified folder, performs gender detection, and saves the result.

In [6]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import cvlib as cv
import os

def process_images_in_folder(source_folder, destination_folder):
    # Load the gender detection model
    model = load_model('gender_detection_largeDataset_v2.model')

    # Define the classes for gender labels
    classes = ['man', 'woman']

    # Iterate through all files in the source folder
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_image_path = os.path.join(source_folder, filename)

            # Read the input image
            image = cv2.imread(input_image_path)

            # Apply face detection using cvlib
            faces, confidences = cv.detect_face(image)

            # Loop through detected faces
            for idx, face in enumerate(faces):
                # Get corner points of face rectangle
                (startX, startY) = face[0], face[1]
                (endX, endY) = face[2], face[3]

                # Crop the detected face region from the image
                face_crop = np.copy(image[startY:endY, startX:endX])

                # If the cropped face region is too small, skip further processing
                if face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
                    continue

                # Preprocess the cropped face for gender detection
                face_crop = cv2.resize(face_crop, (96, 96))
                face_crop = face_crop.astype("float") / 255.0
                face_crop = img_to_array(face_crop)
                face_crop = np.expand_dims(face_crop, axis=0)

                # Apply gender detection on face using the loaded model
                conf = model.predict(face_crop)[0]

                # Get the predicted gender label and confidence score
                idx = np.argmax(conf)
                label = classes[idx]
                confidence = conf[idx] * 100

                # Draw a rectangle around the detected face
                cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

                # Add text label above the face rectangle
                label_text = f"{label.capitalize()} ({confidence:.2f}%)"
                cv2.putText(image, label_text, (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Save the output image to the destination folder
            output_image_path = os.path.join(destination_folder, filename)
            cv2.imwrite(output_image_path, image)

# Example usage
source_folder = r'.\test_images'
destination_folder = r'.\output_images'
process_images_in_folder(source_folder, destination_folder)


1/1 [==============================] - 0s 39ms/step


Processes a batch of images in a specified folder, performs gender detection, and dominant top wear color detection and saves the result.

In [7]:
def process_images_in_folder(source_folder, destination_folder):
    # Load the gender detection model
    gender_model = load_model('gender_detection_largeDataset_v2.model')

    # Iterate through all files in the source folder
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_image_path = os.path.join(source_folder, filename)

            # Read the input image
            frame = cv2.imread(input_image_path)

            # Apply face detection using cvlib
            faces, confidences = cv.detect_face(frame)

            for idx, face in enumerate(faces):
                (startX, startY) = face[0], face[1]
                (endX, endY) = face[2], face[3]

                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

                face_crop = np.copy(frame[startY:endY, startX:endX])

                # Crop a region just below the detected face (adjusting it lower and wider)
                dress_region = frame[endY + 100:endY + 250, startX:endX]

                if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
                    continue

                face_crop = cv2.resize(face_crop, (96, 96))
                face_crop = face_crop.astype("float") / 255.0
                face_crop = img_to_array(face_crop)
                face_crop = np.expand_dims(face_crop, axis=0)

                gender_conf = gender_model.predict(face_crop)[0]
                gender_label = "Woman" if gender_conf[1] > gender_conf[0] else "Man"
                gender_percentage = max(gender_conf) * 100

                # Calculate the color histogram in the dress region
                hist = cv2.calcHist([dress_region], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
                dominant_color_bin = np.unravel_index(hist.argmax(), hist.shape)
                dominant_color_bgr = [int(b * 32) for b in dominant_color_bin]

                # Display the gender and dress color with percentage and color name near each face
                gender_text = f"Gender: {gender_label} ({gender_percentage:.2f}%)"
                color_text = f"Dress Color: {dominant_color_bgr}"

                labelY = startY - 10 if startY - 10 > 10 else startY + 10

                cv2.putText(frame, gender_text, (startX, labelY), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, color_text, (startX, labelY + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, dominant_color_bgr, 2)

                # Draw a rectangle around the dress color region
                cv2.rectangle(frame, (startX, endY + 100), (endX, endY + 250), dominant_color_bgr, 2)

            # Save the resulting image with information to the destination folder
            output_path = os.path.join(destination_folder, f"result_{filename}")
            cv2.imwrite(output_path, frame)
            print(f"Result saved to: {output_path}")

# Example usage
source_folder = r'.\test_images'
destination_folder = r'.\output_images_2'
process_images_in_folder(source_folder, destination_folder)


1/1 [==============================] - 0s 66ms/step
Result saved to: .\output_images_2\result_-little-indian-girls-ready-brush-teeth-isolated-green-background_466689-8858.jpg
1/1 [==============================] - 0s 73ms/step
Result saved to: .\output_images_2\result_IMG-20240103-WA0004.jpg
1/1 [==============================] - 0s 79ms/step
Result saved to: .\output_images_2\result_IMG-20240103-WA0005.jpg
1/1 [==============================] - 0s 71ms/step
Result saved to: .\output_images_2\result_IMG-20240121-WA0021.jpg
1/1 [==============================] - 0s 70ms/step
Result saved to: .\output_images_2\result_IMG-20240121-WA0024.jpg
1/1 [==============================] - 0s 40ms/step
Result saved to: .\output_images_2\result_IMG-20240121-WA0032.jpg
1/1 [==============================] - 0s 44ms/step
Result saved to: .\output_images_2\result_indian-boys-posing-photo-village-near-hampi-india-39234734.jpg
1/1 [==============================] - 0s 39ms/step
Result saved to: .\output_